## Import the Libraries

In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from torchvision.transforms import Lambda, ToTensor

## Write the Datasets and Dataloaders

In [2]:
# Create a Custom Dataset from train_data.csv and eval_data.csv
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pandas.read_csv(csv_file)
        self.transform = transform
        # Remove the first row from the Dataframe
        self.data = self.data.iloc[1:]

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist() # Convert the tensor to a list
        # Get the data from the Dataframe
        data = self.data.iloc[idx]
        # Convert the data to a numpy array
        data = data[0] # First column of the Dataframe
        data_num = []
        for i in data:
            data_num.append(ord(i)- 96)
        data = np.array(data_num)
        # Convert the data to a tensor
        data = torch.from_numpy(data)
        label = self.data.iloc[idx]
        label = label[1]
        label_num = []
        for i in label:
            label_num.append(ord(i)- 96)
        # Append a 0 to the label_num list at the beginning
        label_num.insert(0, 0)
        label = np.array(label_num)
        if self.transform:
            data = self.transform(data) # Apply the transform on the data
        return data, label

# Write a DataLoader for the Custom Dataset
train_dataset = CustomDataset('./A3 files/train_data.csv', transform=None)

# Take only the first 200 examples
# train_dataset.data = train_dataset.data.iloc[:200]

# Split the Dataset into Train and Validation Datasets
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.2)

# Create a DataLoader for the Train and Validation Datasets
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=True)

############################# TESTING CODE #############################

# Print out some data from the Train Dataset
for i, data in enumerate(train_dataloader):
    print(data)
    break

[tensor([[ 1, 15, 19,  5, 13,  5, 12, 22]]), tensor([[ 0, 15, 20, 24, 12, 16, 13, 24, 25]])]


## Write the Transformer Class

In [3]:
import math

class PositionalEncoding(nn.Module):
    # Injecting Some Information about the Relative or the Absolute Positioning of the tokens in the sequence
    def __init__(self, d_model, dropout = 0.1, max_len=8):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p = dropout)
        position = []
        pos_enc = torch.zeros(max_len, d_model) # Positional Encoding -> Max Length and the dimensions of the model
        for i in range(max_len):
            position.append(i)
        position = torch.tensor(position).unsqueeze(1) # Got the pos -> value
        # Now, you want to make the position term to be max_len, d_model
        position_stacked = [position] * d_model 
        position = torch.cat(position_stacked, dim=1)
        # Now to obtain the 10000^2i/d_model 
        div_term = torch.arange(0, d_model, 2) # The Value to be divided
        div_term = div_term/d_model
        div_term = div_term.type(torch.float64)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))   
        pos_enc[:, 0::2] = torch.sin(position[:, 0::2]*div_term)
        pos_enc[:, 1::2] = torch.cos(position[:, 1::2]*div_term)
        self.pos_enc = pos_enc
        return
    
    def forward(self, x):
        x = x + self.pos_enc # This is placeholder -> Modify to include only the len given -> Not Max Len
        #print("Positional Encoding Completed")
        return x

class PosEncoding(nn.Module):
    def __init__(self):
        super(PosEncoding, self).__init__()
        self.max_len = 1000
        self.embedding_dim = 400
        #self.embedding_layer = nn.Embedding(len(word2index), self.embedding_dim)
    def forward(self, embedding):
        # embeddings (b, s, d)
        #embedding = self.embedding_layer(x)
        pos_max = embedding.shape[1]
        d_max = embedding.shape[2]
        # create pos encoding matrix of size (b, s, d)
        pos_encoding = torch.zeros((pos_max, d_max))
        for pos in range(pos_max):
            for i in range(0, d_max, 2):
                pos_encoding[pos, i] = np.sin(pos / (10000 ** ((2 * i)/d_max)))
                pos_encoding[pos, i+1] = np.cos(pos / (10000 ** ((2 * (i+1))/d_max)))
        pos_encoding = pos_encoding.unsqueeze(0)
        pos_encoding = pos_encoding.repeat(embedding.shape[0], 1, 1)
        #print(embedding.shape, pos_encoding.shape)
        embedding = embedding + pos_encoding.to(embedding.device)
        return embedding
# Create a Transformer Encoder
class TransformerEncoder(nn.Module):
    def __init__(self, d_model, nhead, num_layers, dim_feedforward, dropout=0.1, activation="relu"):
        super(TransformerEncoder, self).__init__()
        self.d_model = d_model
        self.nhead = nhead
        self.num_layers = num_layers
        self.dim_feedforward = dim_feedforward
        self.dropout = dropout
        self.activation = activation
        self.pos_enc = PositionalEncoding(d_model, dropout, max_len = 8)
        self.transformer_encoder = nn.TransformerEncoder(nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, activation), num_layers)
        return
    
    def forward(self, x):
        x = self.pos_enc(x)
        x = self.transformer_encoder(x)
        return x

# Create a Transformer Decoder
class TransformerDecoder(nn.Module):
    def __init__(self, d_model, nhead, num_layers, dim_feedforward, dropout=0.1, activation="relu"):
        super(TransformerDecoder, self).__init__()
        self.d_model = d_model
        self.nhead = nhead
        self.num_layers = num_layers
        self.dim_feedforward = dim_feedforward
        self.dropout = dropout
        self.activation = activation
        self.pos_enc = PosEncoding()
        self.transformer_decoder = nn.TransformerDecoder(nn.TransformerDecoderLayer(d_model, nhead, dim_feedforward, dropout, activation), num_layers)
        self.linear = nn.Linear(d_model, 27)
        return
    
    def forward(self, target, memory):
        #print("Decoder")
        seq_len = target.shape[1]
        target = self.pos_enc(target)
        # add batcg dim to target mask, batch dim = 1
        target_mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1)
        # reshape target and memory
        target = target.permute(1, 0, 2)
        memory = memory.permute(1, 0, 2)
        x = self.transformer_decoder(target, memory, tgt_mask = target_mask)
        x = self.linear(x)
        return x

# Create a Transformer Model
class Transformer(nn.Module):
    def __init__(self, d_model, nhead, num_layers, dim_feedforward, dropout=0.1, activation="relu"):
        super(Transformer, self).__init__()
        self.embedding = nn.Embedding(27, d_model)
        self.embed_outputs = nn.Embedding(27, d_model)
        self.encoder = TransformerEncoder(d_model, nhead, num_layers, dim_feedforward, dropout, activation)
        self.decoder = TransformerDecoder(d_model, nhead, num_layers, dim_feedforward, dropout, activation)
        return
    
    def forward(self, x, y, train=True):
        if train == True:
            x = self.embedding(x) # X = batch_size, seq_len, d_model
            x = self.encoder(x) 
            y = self.embed_outputs(y)
            y = self.decoder(y, x)
        else:
            x = self.embedding(x)
            x = self.encoder(x)
            # Now pass the output of the encoder to the decoder along with the start token
            start_token = torch.tensor([0])
            start_token = start_token.unsqueeze(0)
            # print("Start Token Shape: ", start_token.shape)
            for i in range(8):
                input1 = self.embed_outputs(start_token)
                y = self.decoder(input1, x)
                y = torch.argmax(y, dim=2)
                y = y[ -1, :]
                y = y.unsqueeze(0)
                start_token = torch.cat((start_token, y), dim=1)
            y = start_token
        return y

### Train the Model

In [4]:
# Create a Transformer Model
Model = Transformer(300, 2, 2, 300, 0.1, "relu")

# Create a Loss Function
criterion = nn.CrossEntropyLoss()

# Create an Optimizer
optimizer = torch.optim.Adam(Model.parameters(), lr=0.0001)

# Create a Training Loop
acc_tot = 0
tot_cnt = 0
def train_loop(dataloader, model, loss_fn, optimizer):
    acc_tot = 0
    tot_cnt = 0
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute the prediction and the loss
        pred = model(X, y, train=True)
        # reshape preds to 1 2 0
        pred = pred.permute(1,0,2)
        pred = pred[:, :-1, :]
        y = y[:, 1:]
        #print(pred.shape, y.shape)
        pred = pred.permute(0, 2, 1)
        #print(pred.shape, y.shape)
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # calculate accuracy
        accuracy = (pred.argmax(1) == y).type(torch.float).sum().item()
        # divide by batch size and number of chars
        accuracy = accuracy/(y.shape[0]*y.shape[1])
        acc_tot += accuracy
        tot_cnt += 1
        acc_here = acc_tot/tot_cnt
        # decode the preds using argmax
        prediction = pred.argmax(1)
        acc_here = acc_tot/tot_cnt
        # print("Input")
        # print(X)
        # print("Actual")
        # print(y)
        # print("Label")
        # print(prediction)
        # print("-------------------------------------------------------")
        # Print the loss
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            # print epoch, loss, batch, acc_here
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}] accuracy: {acc_here:>7f}")
            
# Train the Model
epochs = 1

for t in range(2):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, Model, criterion, optimizer)

Epoch 1
-------------------------------
loss: 3.374878  [    0/ 5599] accuracy: 0.000000


/opt/homebrew/lib/python3.11/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


loss: 3.394493  [  100/ 5599] accuracy: 0.069307
loss: 2.155067  [  200/ 5599] accuracy: 0.125000
loss: 2.486464  [  300/ 5599] accuracy: 0.177741
loss: 1.930989  [  400/ 5599] accuracy: 0.232544
loss: 1.705398  [  500/ 5599] accuracy: 0.293413
loss: 1.561754  [  600/ 5599] accuracy: 0.356281
loss: 0.776073  [  700/ 5599] accuracy: 0.406740
loss: 0.536523  [  800/ 5599] accuracy: 0.454276
loss: 1.134993  [  900/ 5599] accuracy: 0.500139
loss: 0.341447  [ 1000/ 5599] accuracy: 0.535964
loss: 0.394169  [ 1100/ 5599] accuracy: 0.571185
loss: 0.069830  [ 1200/ 5599] accuracy: 0.600541
loss: 0.061981  [ 1300/ 5599] accuracy: 0.625000
loss: 0.311970  [ 1400/ 5599] accuracy: 0.647662
loss: 0.054717  [ 1500/ 5599] accuracy: 0.665806
loss: 0.035295  [ 1600/ 5599] accuracy: 0.684338
loss: 0.126707  [ 1700/ 5599] accuracy: 0.700176
loss: 0.117031  [ 1800/ 5599] accuracy: 0.715019
loss: 0.502866  [ 1900/ 5599] accuracy: 0.728367
loss: 0.019121  [ 2000/ 5599] accuracy: 0.739568
loss: 0.162798  [ 21

In [18]:
# Create a Validation Loop  
def val_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    acc_sum = 0
    cnt = 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X, y, train=True)
            pred = pred.permute(1,0,2)
            pred = pred[:, :-1, :]
            y = y[:, 1:]
            pred = pred.permute(0, 2, 1)
            accuracy = (pred.argmax(1) == y).type(torch.float).sum().item()
            accuracy = accuracy/(y.shape[0]*y.shape[1])
            acc_sum += accuracy
            cnt += 1
    test_loss /= num_batches
    acc_here = acc_sum/cnt
    print("Eval loop: loss: ", test_loss, " accuracy: ", acc_here)

val_loop(val_dataloader, Model, criterion)

Eval loop: loss:  0.0  accuracy:  0.9980357142857142


## 

In [19]:
# Run on the Eval Set as well
eval_dataset = CustomDataset('./A3 files/eval_data.csv', transform=None)

# Take only the first 200 examples
# eval_dataset.data = eval_dataset.data.iloc[:200]

# Create a DataLoader for the Eval Dataset
eval_dataloader = DataLoader(eval_dataset, batch_size=1, shuffle=True)

val_loop(eval_dataloader, Model, criterion)


Eval loop: loss:  0.0  accuracy:  0.9979364682341171
